<a href="https://colab.research.google.com/github/Hironobu-Kawaguchi/kmnist/blob/master/colab_kmnist_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KMNISTデータ解析
https://github.com/rois-codh/kmnist

## ライブラリのインポート

In [0]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as dataset
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

## バージョン確認

In [2]:
import sys
print('Python:', sys.version)
print('NumPy:', np.__version__)
print('PyTorch:', torch.__version__)
import torchvision
print('torchvision:', torchvision.__version__)

Python: 3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]
NumPy: 1.17.4
PyTorch: 1.3.1
torchvision: 0.4.2


## GPU

In [3]:
!nvidia-smi

Sat Jan  4 11:03:02 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

# Config

In [0]:
seed = 1
batch_size = 64
epochs = 14
test_batch_size = 1000
lr = 1.0
gamma = 0.7

use_cuda = torch.cuda.is_available()

torch.manual_seed(seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

# KMNIST Dataset

In [0]:
train_loader = torch.utils.data.DataLoader(
    datasets.KMNIST('sample_data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        # transforms.Normalize((0.1307,), (0.3081,))
                        transforms.Normalize((0.1904,), (0.3475,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.KMNIST('sample_data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        # transforms.Normalize((0.1307,), (0.3081,))
                        transforms.Normalize((0.1904,), (0.3475,))
                    ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

## MODEL定義

### Simple CNN
Based on - https://github.com/pytorch/examples/blob/master/mnist/main.py

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        # x = F.relu(x)   # add
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

### ResNet
Based on - https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

In [0]:
def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class BasicBlock(nn.Module):
    expansion = 1
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(BasicBlock, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        if groups != 1 or base_width != 64:
            raise ValueError('BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError("Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = norm_layer(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = norm_layer(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4
    __constants__ = ['downsample']

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1, norm_layer=None):
        super(Bottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        width = int(planes * (base_width / 64.)) * groups
        # Both self.conv2 and self.downsample layers downsample the input when stride != 1
        self.conv1 = conv1x1(inplanes, width)
        self.bn1 = norm_layer(width)
        self.conv2 = conv3x3(width, width, stride, groups, dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = conv1x1(width, planes * self.expansion)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out


class ResNet(nn.Module):

    # def __init__(self, block, layers, num_classes=1000, zero_init_residual=False,
    def __init__(self, block, layers, num_classes=10, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None,
                 norm_layer=None):
        super(ResNet, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d
        self._norm_layer = norm_layer

        # self.inplanes = 64
        self.inplanes = 28
        self.dilation = 1
        if replace_stride_with_dilation is None:
            # each element in the tuple indicates if we should replace
            # the 2x2 stride with a dilated convolution instead
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        # self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3,
        self.conv1 = nn.Conv2d(1, self.inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = norm_layer(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        # Zero-initialize the last BN in each residual branch,
        # so that the residual branch starts with zeros, and each residual block behaves like an identity.
        # This improves the model by 0.2~0.3% according to https://arxiv.org/abs/1706.02677
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        norm_layer = self._norm_layer
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation, norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation,
                                norm_layer=norm_layer))

        return nn.Sequential(*layers)

    def _forward_impl(self, x):
        # See note [TorchScript super()]
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def forward(self, x):
        return self._forward_impl(x)

### VGG
Based on - https://github.com/kkweon/mnist-competition

In [0]:
class VGG(nn.Module):  
    """
    Based on - https://github.com/kkweon/mnist-competition
    """
    def two_conv_pool(self, in_channels, f1, f2):
        s = nn.Sequential(
            nn.Conv2d(in_channels, f1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f1),
            nn.ReLU(inplace=True),
            nn.Conv2d(f1, f2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        for m in s.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        return s
    
    def three_conv_pool(self,in_channels, f1, f2, f3):
        s = nn.Sequential(
            nn.Conv2d(in_channels, f1, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f1),
            nn.ReLU(inplace=True),
            nn.Conv2d(f1, f2, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f2),
            nn.ReLU(inplace=True),
            nn.Conv2d(f2, f3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(f3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        for m in s.children():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
        return s
        
    
    def __init__(self, num_classes=10):
        super(VGG, self).__init__()
        self.l1 = self.two_conv_pool(1, 64, 64)
        self.l2 = self.two_conv_pool(64, 128, 128)
        self.l3 = self.three_conv_pool(128, 256, 256, 256)
        self.l4 = self.three_conv_pool(256, 256, 256, 256)
        
        self.classifier = nn.Sequential(
            nn.Dropout(p = 0.5),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(inplace=True),
            nn.Dropout(p = 0.5),
            nn.Linear(512, num_classes),
        )
    
    def forward(self, x):
        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return F.log_softmax(x, dim=1)

# Train & Test function

In [0]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [0]:
def test(model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Train Epoch: {} Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        epoch, test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

# Model Train

## Train Simple CNN Model

In [11]:
model_cnn = Net().to(device)
model_cnn.eval() 

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [12]:
%%time
optimizer = optim.Adadelta(model_cnn.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model_cnn, device, train_loader, optimizer, epoch)
    test(model_cnn, device, test_loader, epoch)
    scheduler.step()
torch.save(model_cnn.state_dict(), "kmnist_cnn.pt")

Train Epoch: 1 Test set: Average loss: 0.3269, Accuracy: 9120/10000 (91.20%)
Train Epoch: 2 Test set: Average loss: 0.2497, Accuracy: 9327/10000 (93.27%)
Train Epoch: 3 Test set: Average loss: 0.2196, Accuracy: 9443/10000 (94.43%)
Train Epoch: 4 Test set: Average loss: 0.2181, Accuracy: 9460/10000 (94.60%)
Train Epoch: 5 Test set: Average loss: 0.2303, Accuracy: 9479/10000 (94.79%)
Train Epoch: 6 Test set: Average loss: 0.2322, Accuracy: 9469/10000 (94.69%)
Train Epoch: 7 Test set: Average loss: 0.2475, Accuracy: 9465/10000 (94.65%)
Train Epoch: 8 Test set: Average loss: 0.2368, Accuracy: 9484/10000 (94.84%)
Train Epoch: 9 Test set: Average loss: 0.2267, Accuracy: 9503/10000 (95.03%)
Train Epoch: 10 Test set: Average loss: 0.2268, Accuracy: 9492/10000 (94.92%)
Train Epoch: 11 Test set: Average loss: 0.2416, Accuracy: 9493/10000 (94.93%)
Train Epoch: 12 Test set: Average loss: 0.2338, Accuracy: 9491/10000 (94.91%)
Train Epoch: 13 Test set: Average loss: 0.2364, Accuracy: 9494/10000 (94.

## Train ResNet18

In [13]:
model_resnet18 = ResNet(BasicBlock, [2, 2, 2, 2]).to(device)
model_resnet18.eval() 

ResNet(
  (conv1): Conv2d(1, 28, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(28, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (downsample): Sequential(
        (0): Conv2d(28, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): BasicBlock(
      (conv1)

In [14]:
%%time
optimizer = optim.Adadelta(model_resnet18.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model_resnet18, device, train_loader, optimizer, epoch)
    test(model_resnet18, device, test_loader, epoch)
    scheduler.step()
torch.save(model_resnet18.state_dict(), "kmnist_resnet18.pt")

Train Epoch: 1 Test set: Average loss: 0.2490, Accuracy: 9266/10000 (92.66%)
Train Epoch: 2 Test set: Average loss: 0.1459, Accuracy: 9619/10000 (96.19%)
Train Epoch: 3 Test set: Average loss: 0.1410, Accuracy: 9672/10000 (96.72%)
Train Epoch: 4 Test set: Average loss: 0.1499, Accuracy: 9680/10000 (96.80%)
Train Epoch: 5 Test set: Average loss: 0.1386, Accuracy: 9726/10000 (97.26%)
Train Epoch: 6 Test set: Average loss: 0.1465, Accuracy: 9718/10000 (97.18%)
Train Epoch: 7 Test set: Average loss: 0.1485, Accuracy: 9724/10000 (97.24%)
Train Epoch: 8 Test set: Average loss: 0.1529, Accuracy: 9733/10000 (97.33%)
Train Epoch: 9 Test set: Average loss: 0.1502, Accuracy: 9730/10000 (97.30%)
Train Epoch: 10 Test set: Average loss: 0.1477, Accuracy: 9739/10000 (97.39%)
Train Epoch: 11 Test set: Average loss: 0.1502, Accuracy: 9727/10000 (97.27%)
Train Epoch: 12 Test set: Average loss: 0.1491, Accuracy: 9740/10000 (97.40%)
Train Epoch: 13 Test set: Average loss: 0.1532, Accuracy: 9730/10000 (97.

## Train ResNet50

In [15]:
model_resnet50 = ResNet(Bottleneck, [3, 4, 6, 3]).to(device)     # ResNet50
model_resnet50.eval() 

ResNet(
  (conv1): Conv2d(1, 28, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(28, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(28, 256, kernel_size=(1, 1), stride=(1, 

In [16]:
%%time
optimizer = optim.Adadelta(model_resnet50.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model_resnet50, device, train_loader, optimizer, epoch)
    test(model_resnet50, device, test_loader, epoch)
    scheduler.step()
torch.save(model_resnet50.state_dict(), "kmnist_resnet50.pt")

Train Epoch: 1 Test set: Average loss: 0.2617, Accuracy: 9226/10000 (92.26%)
Train Epoch: 2 Test set: Average loss: 0.1573, Accuracy: 9575/10000 (95.75%)
Train Epoch: 3 Test set: Average loss: 0.1656, Accuracy: 9577/10000 (95.77%)
Train Epoch: 4 Test set: Average loss: 0.1590, Accuracy: 9637/10000 (96.37%)
Train Epoch: 5 Test set: Average loss: 0.1570, Accuracy: 9669/10000 (96.69%)
Train Epoch: 6 Test set: Average loss: 0.1774, Accuracy: 9675/10000 (96.75%)
Train Epoch: 7 Test set: Average loss: 0.1868, Accuracy: 9682/10000 (96.82%)
Train Epoch: 8 Test set: Average loss: 0.1879, Accuracy: 9674/10000 (96.74%)
Train Epoch: 9 Test set: Average loss: 0.1881, Accuracy: 9678/10000 (96.78%)
Train Epoch: 10 Test set: Average loss: 0.1923, Accuracy: 9679/10000 (96.79%)
Train Epoch: 11 Test set: Average loss: 0.1963, Accuracy: 9676/10000 (96.76%)
Train Epoch: 12 Test set: Average loss: 0.1966, Accuracy: 9678/10000 (96.78%)
Train Epoch: 13 Test set: Average loss: 0.1979, Accuracy: 9677/10000 (96.

## Train ResNeXt50

In [17]:
kwargs['groups'] = 32
kwargs['width_per_group'] = 4
model_resnext50 = ResNet(Bottleneck, [3, 4, 6, 3]).to(device)     # resnext50_32x4d
model_resnext50.eval()

ResNet(
  (conv1): Conv2d(1, 28, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(28, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(28, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(28, 256, kernel_size=(1, 1), stride=(1, 

In [18]:
%%time
optimizer = optim.Adadelta(model_resnext50.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model_resnext50, device, train_loader, optimizer, epoch)
    test(model_resnext50, device, test_loader, epoch)
    scheduler.step()
torch.save(model_resnext50.state_dict(), "kmnist_resnext50.pt")

Train Epoch: 1 Test set: Average loss: 0.3556, Accuracy: 8933/10000 (89.33%)
Train Epoch: 2 Test set: Average loss: 0.1652, Accuracy: 9542/10000 (95.42%)
Train Epoch: 3 Test set: Average loss: 0.1378, Accuracy: 9623/10000 (96.23%)
Train Epoch: 4 Test set: Average loss: 0.1529, Accuracy: 9637/10000 (96.37%)
Train Epoch: 5 Test set: Average loss: 0.1521, Accuracy: 9669/10000 (96.69%)
Train Epoch: 6 Test set: Average loss: 0.1621, Accuracy: 9684/10000 (96.84%)
Train Epoch: 7 Test set: Average loss: 0.1651, Accuracy: 9687/10000 (96.87%)
Train Epoch: 8 Test set: Average loss: 0.1648, Accuracy: 9696/10000 (96.96%)
Train Epoch: 9 Test set: Average loss: 0.1732, Accuracy: 9695/10000 (96.95%)
Train Epoch: 10 Test set: Average loss: 0.1721, Accuracy: 9700/10000 (97.00%)
Train Epoch: 11 Test set: Average loss: 0.1763, Accuracy: 9697/10000 (96.97%)
Train Epoch: 12 Test set: Average loss: 0.1723, Accuracy: 9701/10000 (97.01%)
Train Epoch: 13 Test set: Average loss: 0.1738, Accuracy: 9699/10000 (96.

## Train VGG

In [19]:
# model_vgg = VGG(make_layers(cfgs['A'], batch_norm=False), num_classes=10).to(device)
model_vgg = VGG().to(device)
model_vgg.eval() 

VGG(
  (l1): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (l2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (l3):

In [20]:
%%time
optimizer = optim.Adadelta(model_vgg.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model_vgg, device, train_loader, optimizer, epoch)
    test(model_vgg, device, test_loader, epoch)
    scheduler.step()
torch.save(model_vgg.state_dict(), "kmnist_vgg.pt")

Train Epoch: 1 Test set: Average loss: 0.2524, Accuracy: 9317/10000 (93.17%)
Train Epoch: 2 Test set: Average loss: 0.1576, Accuracy: 9618/10000 (96.18%)
Train Epoch: 3 Test set: Average loss: 0.1479, Accuracy: 9693/10000 (96.93%)
Train Epoch: 4 Test set: Average loss: 0.1180, Accuracy: 9753/10000 (97.53%)
Train Epoch: 5 Test set: Average loss: 0.1010, Accuracy: 9794/10000 (97.94%)
Train Epoch: 6 Test set: Average loss: 0.0968, Accuracy: 9812/10000 (98.12%)
Train Epoch: 7 Test set: Average loss: 0.0956, Accuracy: 9816/10000 (98.16%)
Train Epoch: 8 Test set: Average loss: 0.1018, Accuracy: 9812/10000 (98.12%)
Train Epoch: 9 Test set: Average loss: 0.0974, Accuracy: 9814/10000 (98.14%)
Train Epoch: 10 Test set: Average loss: 0.1000, Accuracy: 9809/10000 (98.09%)
Train Epoch: 11 Test set: Average loss: 0.1018, Accuracy: 9809/10000 (98.09%)
Train Epoch: 12 Test set: Average loss: 0.0983, Accuracy: 9812/10000 (98.12%)
Train Epoch: 13 Test set: Average loss: 0.1003, Accuracy: 9813/10000 (98.

# Ensemble of ResNeXt50 & VGG

In [0]:
class VGG_ResNeXt(nn.Module):
    def __init__(self):
        super(VGG_ResNeXt, self).__init__()
        self.vgg = VGG()
        kwargs['groups'] = 32
        kwargs['width_per_group'] = 4
        self.resnext50 = ResNet(Bottleneck, [3, 4, 6, 3])
    
    def forward(self, x):
        vgg_out = self.vgg(x)
        resnext_out = self.resnext50(x)
        out = (vgg_out + resnext_out) / 2
        return out

In [22]:
model_resnext_vgg = VGG_ResNeXt().to(device)
model_resnext_vgg.eval() 

VGG_ResNeXt(
  (vgg): VGG(
    (l1): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (l2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, strid

In [23]:
%%time
optimizer = optim.Adadelta(model_resnext_vgg.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model_resnext_vgg, device, train_loader, optimizer, epoch)
    test(model_resnext_vgg, device, test_loader, epoch)
    scheduler.step()

Train Epoch: 1 Test set: Average loss: 0.2900, Accuracy: 9238/10000 (92.38%)
Train Epoch: 2 Test set: Average loss: 0.1269, Accuracy: 9650/10000 (96.50%)
Train Epoch: 3 Test set: Average loss: 0.0981, Accuracy: 9750/10000 (97.50%)
Train Epoch: 4 Test set: Average loss: 0.0984, Accuracy: 9756/10000 (97.56%)
Train Epoch: 5 Test set: Average loss: 0.1035, Accuracy: 9777/10000 (97.77%)
Train Epoch: 6 Test set: Average loss: 0.0957, Accuracy: 9786/10000 (97.86%)
Train Epoch: 7 Test set: Average loss: 0.0932, Accuracy: 9803/10000 (98.03%)
Train Epoch: 8 Test set: Average loss: 0.0943, Accuracy: 9798/10000 (97.98%)
Train Epoch: 9 Test set: Average loss: 0.0958, Accuracy: 9805/10000 (98.05%)
Train Epoch: 10 Test set: Average loss: 0.0964, Accuracy: 9811/10000 (98.11%)
Train Epoch: 11 Test set: Average loss: 0.0952, Accuracy: 9810/10000 (98.10%)
Train Epoch: 12 Test set: Average loss: 0.0952, Accuracy: 9815/10000 (98.15%)
Train Epoch: 13 Test set: Average loss: 0.0930, Accuracy: 9804/10000 (98.

# Ensemble of ResNet18 & VGG

In [0]:
class VGG_ResNet(nn.Module):
    def __init__(self):
        super(VGG_ResNet, self).__init__()
        self.vgg = VGG()
        self.resnet18 = ResNet(BasicBlock, [2, 2, 2, 2], groups=1, width_per_group=64)
    
    def forward(self, x):
        vgg_out = self.vgg(x)
        resnet_out = self.resnet18(x)
        out = (vgg_out + resnet_out) / 2
        return out

In [25]:
model_resnet_vgg = VGG_ResNet().to(device)
model_resnet_vgg.eval() 

VGG_ResNet(
  (vgg): VGG(
    (l1): Sequential(
      (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (l2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride

In [26]:
%%time
optimizer = optim.Adadelta(model_resnet_vgg.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model_resnet_vgg, device, train_loader, optimizer, epoch)
    test(model_resnet_vgg, device, test_loader, epoch)
    scheduler.step()
torch.save(model_resnet_vgg.state_dict(), "kmnist_resnet_vgg.pt")

Train Epoch: 1 Test set: Average loss: 0.2625, Accuracy: 9270/10000 (92.70%)
Train Epoch: 2 Test set: Average loss: 0.1421, Accuracy: 9618/10000 (96.18%)
Train Epoch: 3 Test set: Average loss: 0.1225, Accuracy: 9703/10000 (97.03%)
Train Epoch: 4 Test set: Average loss: 0.1323, Accuracy: 9717/10000 (97.17%)
Train Epoch: 5 Test set: Average loss: 0.1208, Accuracy: 9757/10000 (97.57%)
Train Epoch: 6 Test set: Average loss: 0.1209, Accuracy: 9750/10000 (97.50%)
Train Epoch: 7 Test set: Average loss: 0.1214, Accuracy: 9762/10000 (97.62%)
Train Epoch: 8 Test set: Average loss: 0.1247, Accuracy: 9748/10000 (97.48%)
Train Epoch: 9 Test set: Average loss: 0.1240, Accuracy: 9746/10000 (97.46%)
Train Epoch: 10 Test set: Average loss: 0.1245, Accuracy: 9748/10000 (97.48%)
Train Epoch: 11 Test set: Average loss: 0.1239, Accuracy: 9758/10000 (97.58%)
Train Epoch: 12 Test set: Average loss: 0.1241, Accuracy: 9756/10000 (97.56%)
Train Epoch: 13 Test set: Average loss: 0.1273, Accuracy: 9751/10000 (97.